### Common imports

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Output, Dropdown, SelectMultiple, HBox, VBox, Button, IntSlider, FloatRangeSlider, FloatSlider, Text, Textarea, Combobox, HBox, VBox, SelectMultiple, Tab
from IPython.display import display

from surianalytics.connectors import RESTSciriusConnector
from surianalytics.datamining import min_max_scaling

import pandas as pd
import numpy as np

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', None)

In [ ]:
c = RESTSciriusConnector()
c.set_page_size(1000)

### Timestamp handlers

In [ ]:
from datetime import datetime, timedelta

In [ ]:
TIME_END = datetime.utcnow()
TIME_BEGINNING = TIME_END - timedelta(minutes=60)

In [ ]:
SLIDER_TIME_MINUTES = IntSlider(
    value=60,
    min=5,
    max=600,
    step=5,
    description='Minutes',
    orientation='horizontal',
    readout=True,
)

In [ ]:
# TimePicker is not in ipywidgests 7.7
# Use text boxes instead for now
TEXT_TIME_BEGINNING = Text(
    description="From: ",
    value=TIME_BEGINNING.isoformat(),
    continuous_update=True
)
TEXT_TIME_END = Text(
    description="To: ",
    value=TIME_END.isoformat(),
    continuous_update=True
)
_ = c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
def handler_update_timeframe(args):
    c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
BUTTON_UPDATE_TIME = Button(description="Set time")
BUTTON_UPDATE_TIME.on_click(handler_update_timeframe)

In [ ]:
def handler_reset_timeframe(args):
    time_to = datetime.utcnow()
    time_from = time_to - timedelta(minutes=SLIDER_TIME_MINUTES.value)
    TEXT_TIME_END.value = time_to.isoformat()
    TEXT_TIME_BEGINNING.value = time_from.isoformat()
    c.set_query_delta(minutes=SLIDER_TIME_MINUTES.value)

In [ ]:
BUTTON_RESET_TIME = Button(description="Generate time")
BUTTON_RESET_TIME.on_click(handler_reset_timeframe)

In [ ]:
import pickle
PKL_TIME = "./time.pkl"

In [ ]:
def handler_pickle_time_save(args):
    with open(PKL_TIME, "wb") as handle:
        pickle.dump((TEXT_TIME_BEGINNING.value, TEXT_TIME_END.value), handle)

In [ ]:
BUTTON_SAVE_TIME = Button(description="Dump time")
BUTTON_SAVE_TIME.on_click(handler_pickle_time_save)

In [ ]:
def handler_pickle_time_load(args):
    handle = open(PKL_TIME, "rb")
    times = pickle.load(handle)
    TEXT_TIME_BEGINNING.value = times[0]
    TEXT_TIME_END.value = times[1]
    handler_update_timeframe(args)

In [ ]:
BUTTON_LOAD_TIME = Button(description="Sync time")
BUTTON_LOAD_TIME.on_click(handler_pickle_time_load)

In [ ]:
BOX_TIME = VBox([
    HBox([SLIDER_TIME_MINUTES, BUTTON_RESET_TIME]), 
    HBox([
        VBox([TEXT_TIME_BEGINNING, TEXT_TIME_END]), 
        BUTTON_UPDATE_TIME,
        VBox([BUTTON_SAVE_TIME, BUTTON_LOAD_TIME])
    ])
])

### Query handler

In [ ]:
TEXT_QUERY = Textarea(
    description='Query filter:',
    value="*",
    continuous_update=True
)

In [ ]:
import pickle
PKL_QUERY = "./query.pkl"

In [ ]:
def handler_pickle_query_save(args):
    with open(PKL_QUERY, "wb") as handle:
        pickle.dump(TEXT_QUERY.value, handle)

In [ ]:
BUTTON_SAVE_QUERY = Button(description="Dump query")
BUTTON_SAVE_QUERY.on_click(handler_pickle_query_save)

In [ ]:
def handler_pickle_query_load(args):
    handle = open(PKL_QUERY, "rb")
    TEXT_QUERY.value = pickle.load(handle)

In [ ]:
BUTTON_LOAD_QUERY = Button(description="Sync query")
BUTTON_LOAD_QUERY.on_click(handler_pickle_query_load)

In [ ]:
BOX_QUERY = HBox([
    TEXT_QUERY, 
    VBox([BUTTON_SAVE_QUERY, BUTTON_LOAD_QUERY])
])

### Explore events

In [ ]:
import copy

In [ ]:
DEFAULT_COLUMNS = ["timestamp", "flow_id", "event_type"]
COLS = copy.deepcopy(DEFAULT_COLUMNS)

In [ ]:
OUTPUT_DEBUG = Output()

In [ ]:
DF_EVENTS = pd.DataFrame(columns=DEFAULT_COLUMNS)

In [ ]:
SELECT_COLUMNS = Combobox(options=sorted(list(DF_EVENTS.dropna(how="all", axis=1).columns.values)), description="Columns")

In [ ]:
SELECT_AGG_COLUMN = Combobox(options=COLS, value="flow_id", description="Group by")

In [ ]:
def handler_add_col(args):
    OUTPUT_DEBUG.clear_output()
    with OUTPUT_DEBUG:
        global COLS
        col = SELECT_COLUMNS.value
        if col in list(DF_EVENTS.columns.values) and col not in COLS:
            COLS.append(col)
            SELECT_AGG_COLUMN.options = COLS
            print(COLS)
        else:
            print("{} already in selected columns".format(col))

In [ ]:
BUTTON_ADD_COL = Button(description="Add column")
BUTTON_ADD_COL.on_click(handler_add_col)

In [ ]:
def handler_clear_col(args):
    OUTPUT_DEBUG.clear_output()
    with OUTPUT_DEBUG:
        global COLS
        COLS = DEFAULT_COLUMNS
        SELECT_AGG_COLUMN.options = DEFAULT_COLUMNS
        print("columns are now {}".format(COLS))

In [ ]:
BUTTON_CLEAR_COL = Button(description="Clear columns")
BUTTON_CLEAR_COL.on_click(handler_clear_col)

In [ ]:
OUTPUT_EVE = Output()

In [ ]:
def handler_pull_events(args):
    OUTPUT_EVE.clear_output()
    with OUTPUT_EVE:
        global DF_EVENTS
        DF_EVENTS = c.get_events_df(qfilter=TEXT_QUERY.value)
        SELECT_COLUMNS.options = list(DF_EVENTS.columns.values)
        print("downloaded {} rows".format(len(DF_EVENTS)))

In [ ]:
BUTTON_EVE_PULL = Button(description="Download EVE")
BUTTON_EVE_PULL.on_click(handler_pull_events)

In [ ]:
def handler_aggregate_events(args):
    OUTPUT_EVE.clear_output()
    with OUTPUT_EVE:
        df = (
            DF_EVENTS
                .sort_values(by="timestamp")
                .dropna(axis=1, how="all")
                .groupby(SELECT_AGG_COLUMN.value)
                .agg({
                    item: ["min", "max"] if item == "timestamp"
                    else [lambda x: x.nunique()] if item == "flow_id"
                    else ["unique"]
                    for item in COLS
                    if item != SELECT_AGG_COLUMN.value
                })
        )
        display(df)

In [ ]:
BUTTON_EVE_AGG = Button(description="Aggregate EVE")
BUTTON_EVE_AGG.on_click(handler_aggregate_events)

In [ ]:
def handler_show_events(args):
    OUTPUT_EVE.clear_output()
    with OUTPUT_EVE:
        df = (
            DF_EVENTS[COLS]
                .sort_values(by="timestamp")
                .head(500)
                .dropna(axis=1, how="all")
        )
        display(df)

In [ ]:
BUTTON_EVE_SHOW = Button(description="Show EVE")
BUTTON_EVE_SHOW.on_click(handler_show_events)

In [ ]:
BOX_EVENTS = VBox([
    SELECT_AGG_COLUMN,
    HBox([SELECT_COLUMNS, BUTTON_ADD_COL, BUTTON_CLEAR_COL]),
    HBox([BUTTON_EVE_PULL, BUTTON_EVE_SHOW, BUTTON_EVE_AGG]),
    OUTPUT_DEBUG,
    OUTPUT_EVE,
])

### Hunting section

In [ ]:
tab = Tab(children=[BOX_TIME, BOX_QUERY, BOX_EVENTS])
tab.set_title(0, "Timepicker")
tab.set_title(1, "Query filter")
tab.set_title(2, "Analytics")

In [ ]:
tab